In [1]:
import os
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [2]:
# Get embeddings.
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

texts = [
    "Brasil propõe fundo ambiental de 12 bi de euros para fechar acordo com UE.",
    "O Itamaraty inverte a lógica da desconfiança e pede a ajuda da Europa para cumprir as metas ambientais.",
    "a melhoria das medidas higiénicas não diminui a prevalência das doenças retrovirais, e a frequência de hospitalizações se mantém alta apesar do uso de terapia de reidratação oral.",
    "a principal intervenção sanitária que se pode aplicar é a vacina contra rotavírus",
    "Hamas diz ter libertado duas refens americanas por razões humanitárias.",
    "O Conselho Europeu (cujas reuniões são também conhecidas por Cimeiras Europeias) é uma instituição da União Europeia de carácter eminentemente político prevista nos artigos 235° e 236° do TFUE.",
    "Antes da Revisão do Tratado de Lisboa, o papel do presidente (presidente em exercício) do Conselho Europeu era exercido pelo Chefe de Governo ou Chefe de Estado do Estado-Membro que exercia a Presidência do Conselho no respectivo semestre",
    "A União Europeia (UE) é uma união económica e política de 27 Estados-membros independentes situados principalmente na Europa.",
    "Rotavirus é um género de vírus de RNA bicatenário da família Reoviridae",
    "O território da Alemanha abrange 357.021 quilômetros quadrados e é influenciado por um clima temperado sazonal.",
    "O nacionalismo é uma ideia e um movimento que defende que a nação deve ser congruente com o Estado.",
    "Os nacionalistas territoriais assumem que todos os habitantes de uma nação em particular devem lealdade ao seu país de nascimento ou adoção.",
    "O rotavírus foi descoberto em 1973 e é responsável por 50% dos internamentos hospitalares devido a diarreias agudas em crianças. Para além do seu impacto na saúde humana, os rotavírus também causam infecções em animais, inclusive no gado.", 
    "Cecilia Payne-Gaposchkin (Wendover, Buckinghamshire, 10 de maio de 1900 — Cambridge (Massachusetts), 7 de dezembro de 1979) foi uma astrônoma e astrofísica britânico-americana.",
]

# Create a retriever
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)
query = "O que se sabe sobre o rotavírus e a proteção contra este?"

# Get relevant documents ordered by relevance score
docs = retriever.get_relevant_documents(query)
docs

In [22]:
# Reorder the documents:
# Less relevant document will be at the middle of the list and more
# relevant elements at beginning / end.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(page_content='a principal intervenção sanitária que se pode aplicar é a vacina contra rotavírus'),
 Document(page_content='Antes da Revisão do Tratado de Lisboa, o papel do presidente (presidente em exercício) do Conselho Europeu era exercido pelo Chefe de Governo ou Chefe de Estado do Estado-Membro que exercia a Presidência do Conselho no respectivo semestre'),
 Document(page_content='Antes da Revisão do Tratado de Lisboa, o papel do presidente (presidente em exercício) do Conselho Europeu era exercido pelo Chefe de Governo ou Chefe de Estado do Estado-Membro que exercia a Presidência do Conselho no respectivo semestre'),
 Document(page_content='Brasil propõe fundo ambiental de 12 bi de euros para fechar acordo com UE.'),
 Document(page_content='O Itamaraty inverte a lógica da desconfiança e pede a ajuda da Europa para cumprir as metas ambientais.'),
 Document(page_content='O Itamaraty inverte a lógica da desconfiança e pede a ajuda da Europa para cumprir as metas ambientais

In [23]:
# Preparamos e executamos uma cadeia de coisas personalizada com documentos reordenados como contexto.

# Substituir Prompts
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
llm = OpenAI()
stuff_prompt_override = """Dado este texto extraído:
-----
{context}
-----
Por favor responda a seguinte pergunta:
{query}"""
prompt = PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

In [24]:
# Instantiate the chain
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=reordered_docs, query=query)

'\n\nO rotavírus foi descoberto em 1973 e é responsável por 50% dos internamentos hospitalares devido a diarreias agudas em crianças. A principal intervenção sanitária que se pode aplicar é a vacina contra rotavírus, que pode ser usada para reduzir a incidência e a gravidade da infecção. Além disso, também é responsável por infecções em animais, inclusive no gado.'